In [38]:
from numba import njit
from random import randint as rndint
from random import random as rnd
import numpy as np
from math import sqrt

In [39]:
@njit
# checks if position is in board
def isInBoard(position, dim):
    x,y = position
    return (0<=x<dim and 0<=y<dim)

@njit
# finds number of queens hits
def totHits(positions):
    hits = 0
    dim = len(positions)
    for i in range(dim):

        # moves up right
        (x,y) = (i+1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x+=1
            y+=1
        # moves down right
        (x,y) = (i+1, positions[i]-1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x+=1
            y-=1
        # moves up left
        (x,y) = (i-1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x-=1
            y+=1
        (x,y) = (i-1, positions[i]-1)
        # moves down left
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x-=1
            y-=1
    return hits

@njit
def evaluate(positions):
    return -totHits(positions)/2

In [40]:
@njit
def crossover(elem1, elem2, crossRatio):
    size = len(elem1)
    if rnd() >= crossRatio:
        return elem1.copy(), elem2.copy()
    crossPoint1 = rndint(0,size-1)
    # crossPoint2 = rndint(0,size-1)
    crossPoint2 = size-1
    while crossPoint1 == crossPoint2:
        crossPoint2 = rndint(0,size-1)
    # if crossPoint1 > crossPoint2:
    #     crossPoint1, crossPoint2 = crossPoint2, crossPoint1
    child1 = np.full(size, -1, np.int_)
    child2 = np.full(size, -1, np.int_)
    for i in range(crossPoint1, crossPoint2+1):
        child1[i] = elem1[i]
        child2[i] = elem2[i]
    free1 = crossPoint2+1
    free2 = crossPoint2+1
    for i in range(crossPoint2+1, crossPoint2+size+1):
        j = i % size
        if elem1[j] not in child2:
            free2 = free2 % size
            child2[free2] = elem1[j]
            free2 += 1
        if elem2[j] not in child1:
            free1 = free1 % size
            child1[free1] = elem2[j]
            free1 += 1
    return child1, child2

In [41]:
@njit
def tweak(positions):
    l = len(positions)
    x = rndint(0,l-1)
    y = rndint(0,l-1)
    while(x==y):
        y = rndint(0,l-1)
    positions[x], positions[y] = positions[y], positions[x]

@njit
#random starting population
def populationInitialize(populationSize, size):
    population = np.empty((populationSize, size), np.int_)
    for i in range(populationSize):
        element = np.arange(size, dtype=np.int_)
        for _ in range(10 * size):
            tweak(element)
        population[i] = element
    return population

@njit
def mutation(elem, mutationRatio):
    if rnd() < mutationRatio:
        tweak(elem)

In [42]:
@njit
def populationScores(flattenedPopulation, populationSize, size):
    population = flattenedPopulation.reshape((populationSize, size))
    scores = np.empty(populationSize, np.int_)
    for i in range(populationSize):
        scores[i] = evaluate(population[i])
    return scores

In [43]:
# rank based selection
@njit
def index(v):
    dis = 1 + (8*v)
    return int((-1+(sqrt(dis)))/2)

@njit
def rankSelection(flattenPopulation, populationSize, size, scores, totRanks):
    population = flattenPopulation.reshape((populationSize, size))
    pop_scores = sorted(zip(population, scores), key=lambda x: x[1])
    selected = np.empty((len(population), size), dtype=np.int_)
    for i in range(len(population)):
        elemi = index(rndint(0, totRanks-1))
        selected[i] = pop_scores[elemi][0]
    return selected

In [62]:
@njit
def geneticAlgorithm(size, populationSize, iterations, crossRatio, mutationRatio, best, score, index):
    population = populationInitialize(populationSize, size)
    scores = populationScores(population.flatten(), populationSize, size)
    bestScore = scores.max()
    indexBestElement = np.where(scores == bestScore)[0][0]
    bestElement = population[indexBestElement]
    totRanks = int(populationSize * (populationSize + 1) / 2)
    for _ in range(iterations):
        selected = rankSelection(population.flatten(), populationSize, size, scores, totRanks)
        childs = np.empty((populationSize, size), dtype = np.int_)
        for i in range(0, populationSize, 2):
            child1, child2 = crossover(selected[i], selected[i+1], crossRatio)
            mutation(child1, mutationRatio)
            mutation(child2, mutationRatio)
            childs[i] = child1
            childs[i+1] = child2
        population = childs
        scores = populationScores(population.flatten(), populationSize, size)
        bestChildScore = scores.max()
        if bestChildScore > bestScore:
            indexBestElement = np.where(scores == bestChildScore)[0][0]
            bestElement = population[indexBestElement]
            bestScore = bestChildScore
        if bestScore == 0:
            break
    score[index] = bestScore

In [63]:
from threading import Thread
def geneticParallel(cores, size, populationSize, iterations, crossRatio, mutationRatio):
    bests = np.empty(cores, np.ndarray)
    scores = np.empty(cores, np.int_)

    threads = [None] * cores
    for i in range(cores):
        threads[i] = Thread(target = geneticAlgorithm,
                            args = (size, populationSize, iterations, crossRatio,
                                    mutationRatio, bests[i], scores, i))
        threads[i].start()
    for i in range(cores):
        threads[i].join()
    print(scores)

In [65]:
from time import perf_counter
size = 20
iterations = 10000
populationSize = 14
crossRatio =0.90
mutationRatio = 0.1
fails = 0
its = 30
cores = 2

start = perf_counter()
for _ in range(its):
    geneticParallel(cores, size, populationSize, iterations, crossRatio, mutationRatio)
end = perf_counter()

print("average time:", (end-start)/its)
print("fail ratio:", int(100*fails/its), "%")

[0 0]
[0 0]
[ 0 -1]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]
average time: 0.125100452233346
fail ratio: 0 %
